In [1]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# Ensure the variables are defined
persist_directory = '../data/chromadb'

collection_name = 'CLIP_slides_collection'

# Initialize ChromaDB client with the existing settings
client = chromadb.PersistentClient(
    path=persist_directory,
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# List all collections in ChromaDB
collections = client.list_collections()
print("Existing collections:")
for collection_n in collections:
    collection = client.get_collection(collection_n)
    print(collection.name)

print ("")

# Retrieve the specific collection by name
collection = client.get_collection(collection_name)
print(f"Using collection: {collection_name}")

Existing collections:
CLIP_slides_collection

Using collection: CLIP_slides_collection


In [39]:
from transformers import CLIPProcessor, CLIPModel

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Function to embed text using CLIP
def embed_text_with_clip(texts):
    inputs = clip_processor(text=texts, return_tensors="pt", padding=True)
    embeddings = clip_model.get_text_features(**inputs)
    return embeddings.detach().numpy()

# Embed the query
query_text = "Who are the founders of airbnb?"
query_embedding = embed_text_with_clip([query_text])  # Returns a NumPy array

search_results = collection.query(
    query_embeddings=query_embedding,  # Use the embedding generated above
    n_results=3  # Retrieve the top 5 most relevant results
)

print(search_results)

{'ids': [['AirBnB_Pitch_Deck_slide18.png', 'AirBnB_Pitch_Deck_slide4.png', 'AirBnB_Pitch_Deck_slide1.png']], 'embeddings': None, 'documents': [['AirBnB_Pitch_Deck_slide18.png', 'AirBnB_Pitch_Deck_slide4.png', 'AirBnB_Pitch_Deck_slide1.png']], 'uris': None, 'data': None, 'metadatas': [[{'presentation': 'AirBnB'}, {'presentation': 'AirBnB'}, {'presentation': 'AirBnB'}]], 'distances': [[114.04544197510567, 115.84703448228008, 118.01697291037031]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [38]:
import os
from dotenv import load_dotenv
from transformers import CLIPProcessor, CLIPModel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Load environment variables from .env file
load_dotenv()
# Retrieve the OpenAI API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Function to embed text using CLIP
def embed_text_with_clip(texts):
    inputs = clip_processor(text=texts, return_tensors="pt", padding=True)
    embeddings = clip_model.get_text_features(**inputs)
    return embeddings.detach().numpy()

# Step 2: Embed the query text
question = "What is the market validation of AirBnB?"

query_embedding = embed_text_with_clip([question])  # Returns a NumPy array

# Step 3: Query the ChromaDB collection
search_results = collection.query(
    query_embeddings=query_embedding,  # Use the embedding generated above
    n_results=5  # Retrieve the top 5 most relevant results
)

# Step 4: Extract context from the search results
context = "\n".join([doc for sublist in search_results["documents"] for doc in sublist])

# Step 5: Create a prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="{context}\n\nQuestion: {question}\nAnswer:"
)

# Step 6: Initialize the OpenAI LLM through LangChain
llm = ChatOpenAI(
    model="gpt-4",
    openai_api_key=openai_api_key  # Replace with your actual API key
)

# Step 7: Create an LLMChain with the prompt template
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Step 8: Run the LLMChain to generate the answer
response = llm_chain.run(context=context, question=question)

# Step 9: Print the response
print("Response:", response)


Response: The text doesn't provide information on the market validation of AirBnB.
